### --[Day 11: Seating System](https://adventofcode.com/2020/day/11)--

Rules:

    If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
    If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
    Otherwise, the seat's state does not change.


In [1]:
INPUT_FILE = 'input_d11.txt'
EXAMPLE_FILE = 'input2_d11.txt'

class Day11:
    def __init__(self, input_file=INPUT_FILE):
        self.seat_map = Day11.read_file(input_file)
        self.count_adjacent()
        self.count_adjacent_v2()
    
    def count_adjacent(self):
        '''
        Creates a matrix of the count of occupied seats immediately adjacent
        to each cell
        '''
        # Create a matrix of 0s the same dimensions as the seat map
        self.adjacent = []
        for row_num in range(len(self.seat_map)):
            self.adjacent.append([0 for _ in range(len(self.seat_map[row_num]))])
        
        for row_num in range(len(self.seat_map)):
            for col_num in range(len(self.seat_map[row_num])):
                if self.seat_map[row_num][col_num] == '#':
                    # when occupied, increment the 8 adjacent cells in self.adjacent
                    self.increment_adjacent(row_num-1, col_num-1)
                    self.increment_adjacent(row_num-1, col_num)
                    self.increment_adjacent(row_num-1, col_num+1)
                    self.increment_adjacent(row_num, col_num-1)
                    self.increment_adjacent(row_num, col_num+1)
                    self.increment_adjacent(row_num+1, col_num-1)
                    self.increment_adjacent(row_num+1, col_num)
                    self.increment_adjacent(row_num+1, col_num+1)                
                    
    def count_adjacent_v2(self):
        '''
        For Part 2, redefines count_adjacent() to consider adjacent seats
        as the first seat encountered in each of the 4 directions
        '''
        # Create a matrix of 0s the same dimensions as the seat map
        self.adjacent_axes = []
        for row_num in range(len(self.seat_map)):
            self.adjacent_axes.append([0 for _ in range(len(self.seat_map[row_num]))])
        
        points = []
        for row_num in range(len(self.seat_map)):
            for col_num in range(len(self.seat_map[row_num])):
                if self.seat_map[row_num][col_num] == '#':
                    # when occupied, find the 8 adjacent cells
                    points.append(self.find_adjacent(row_num, col_num, -1, -1))
                    points.append(self.find_adjacent(row_num, col_num, -1, 0))
                    points.append(self.find_adjacent(row_num, col_num, -1, 1))
                    points.append(self.find_adjacent(row_num, col_num, 0, -1))
                    points.append(self.find_adjacent(row_num, col_num, 0, 1))
                    points.append(self.find_adjacent(row_num, col_num, 1, -1))
                    points.append(self.find_adjacent(row_num, col_num, 1, 0))
                    points.append(self.find_adjacent(row_num, col_num, 1, 1))
        
        # Update self.adjacent_axes for any seats found
        for row, col in points:
            if row is not None:
                self.adjacent_axes[row][col] += 1

    
    def find_adjacent(self, start_row, start_col, d_row, d_col):
        '''
        Searches along an axis until the next seat is found
        :param start_row: row to search from
        :param start_col: col to search from
        :param d_row: each step will increase or decrease row by this amt
        :return: (row,col) of next seat found in the specified direction, or None if there isn't one
        '''
        row = start_row + d_row
        col = start_col + d_col
        
        while row >= 0 and row < len(self.seat_map) \
            and col >= 0 and col < len(self.seat_map[row]):
            
            if self.seat_map[row][col] != '.':
                return (row, col)
            row += d_row
            col += d_col
        
        return (None, None)
    
    def increment_adjacent(self, row, col):
        '''
        Adds 1 to the adjacent count matrix where specified,
        if the cell to be updated is not out of bounds
        :param row: row to be updated
        :param col: column to be updated
        '''
        if row >= 0 and row < len(self.seat_map) and col >= 0 and col < len(self.seat_map[row]):
            self.adjacent[row][col] += 1
            
    def run_round(self):
        '''
        Updates all applicable seats in the seat map based on the puzzle rules defined:
        * If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
        * If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
        * Otherwise, the seat's state does not change.
        :return: the number of seats that changed state in this round
        '''
        num_changed = 0
        
        for row_num in range(len(self.seat_map)):
            for col_num in range(len(self.seat_map[row_num])):
                if self.seat_map[row_num][col_num] == 'L' and self.adjacent[row_num][col_num] == 0:
                    self.seat_map[row_num][col_num] = '#'
                    num_changed += 1
                elif self.seat_map[row_num][col_num] == '#' and self.adjacent[row_num][col_num] >= 4:
                    self.seat_map[row_num][col_num] = 'L'
                    num_changed += 1
        
        self.count_adjacent()
        return num_changed
    
    def run_round_v2(self):
        '''
        Part 2 - Runs with the modified definitions of the rules
        '''
        num_changed = 0
        
        for row_num in range(len(self.seat_map)):
            for col_num in range(len(self.seat_map[row_num])):
                if self.seat_map[row_num][col_num] == 'L' and self.adjacent_axes[row_num][col_num] == 0:
                    self.seat_map[row_num][col_num] = '#'
                    num_changed += 1
                elif self.seat_map[row_num][col_num] == '#' and self.adjacent_axes[row_num][col_num] >= 5:
                    self.seat_map[row_num][col_num] = 'L'
                    num_changed += 1
        
        self.count_adjacent_v2()
        return num_changed
        
    
    def run_until_stable(self, verbose=False):
        '''
        Runs rounds until seats stop changing state
        :param verbose: prints the results of each round if True
        '''
        round_num = 0
        last_round = None
        while last_round != 0:
            round_num += 1
            last_round = self.run_round()
            if verbose:
                print(f'R{round_num} changed {last_round} seats')
    
    def run_until_stable_v2(self, verbose=False):
        '''
        Runs rounds using the part 2 rules until seats are stable
        :param verbose: prints the results of each round if True
        '''
        round_num = 0
        last_round = None
        while last_round != 0:
            round_num += 1
            last_round = self.run_round_v2()
            if verbose:
                print(f'R{round_num} changed {last_round} seats')
    
    def count_occupied(self):
        '''
        :return: The number of occupied seats on the current map
        '''
        count = 0
        for row in self.seat_map:
            for col in row:
                if col == '#':
                    count += 1
        return count
    
    def print_map(self):
        for i in range(len(self.seat_map)):
            print(f'{i}: {self.seat_map[i]} -> {self.adjacent[i]}')
    
    def print_map_v2(self):
        for i in range(len(self.seat_map)):
            print(f'{i}: {self.seat_map[i]} -> {self.adjacent_axes[i]}')
    
    @staticmethod
    def read_file(input_file):
        result = []
        with open(input_file) as f:
            for row in f.readlines():
                result.append([c for c in row.rstrip()])
        return result
        

In [2]:
# Test against the example data
ex = Day11(EXAMPLE_FILE)
ex.run_until_stable(True)
ex.print_map()
assert ex.count_occupied() == 37

R1 changed 71 seats
R2 changed 51 seats
R3 changed 31 seats
R4 changed 21 seats
R5 changed 7 seats
R6 changed 0 seats
0: ['#', '.', '#', 'L', '.', 'L', '#', '.', '#', '#'] -> [1, 3, 0, 2, 1, 2, 0, 2, 2, 2]
1: ['#', 'L', 'L', 'L', '#', 'L', 'L', '.', 'L', '#'] -> [1, 4, 2, 3, 0, 2, 2, 3, 4, 2]
2: ['L', '.', '#', '.', 'L', '.', '.', '#', '.', '.'] -> [2, 4, 2, 4, 3, 3, 3, 1, 3, 2]
3: ['#', 'L', '#', '#', '.', '#', '#', '.', 'L', '#'] -> [1, 5, 3, 3, 2, 1, 2, 2, 2, 0]
4: ['#', '.', '#', 'L', '.', 'L', 'L', '.', 'L', 'L'] -> [2, 6, 3, 5, 3, 4, 3, 3, 3, 3]
5: ['#', '.', '#', 'L', '#', 'L', '#', '.', '#', '#'] -> [1, 4, 1, 3, 0, 2, 0, 2, 1, 1]
6: ['.', '.', 'L', '.', 'L', '.', '.', '.', '.', '.'] -> [2, 4, 2, 4, 3, 4, 3, 3, 4, 3]
7: ['#', 'L', '#', 'L', '#', '#', 'L', '#', 'L', '#'] -> [1, 3, 0, 2, 1, 1, 2, 0, 2, 0]
8: ['#', '.', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L'] -> [2, 5, 2, 4, 3, 4, 3, 3, 4, 3]
9: ['#', '.', '#', 'L', '#', 'L', '#', '.', '#', '#'] -> [1, 3, 0, 2, 0, 2, 0, 2, 1, 1]


In [3]:
# Part 1 solution
day11 = Day11()
day11.run_until_stable()
day11.count_occupied()

2273

### Part 2

In [4]:
ex = Day11(EXAMPLE_FILE)
ex.print_map_v2()
ex.run_round_v2()
print('*' * 120)
ex.print_map_v2()

0: ['L', '.', 'L', 'L', '.', 'L', 'L', '.', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1: ['L', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2: ['L', '.', 'L', '.', 'L', '.', '.', 'L', '.', '.'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3: ['L', 'L', 'L', 'L', '.', 'L', 'L', '.', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4: ['L', '.', 'L', 'L', '.', 'L', 'L', '.', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5: ['L', '.', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6: ['.', '.', 'L', '.', 'L', '.', '.', '.', '.', '.'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7: ['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8: ['L', '.', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9: ['L', '.', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
************************************************************************************************************************

In [5]:
ex.run_until_stable_v2()
assert ex.count_occupied() == 26

In [6]:
######### Part 2 solution
d11 = Day11()
d11.run_until_stable_v2()
d11.count_occupied()


2064